In [33]:
import torch

In [34]:
!aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz .

/bin/bash: aws: command not found


In [35]:
import pandas as pd
df = pd.read_csv("amazon_reviews_us_Camera_v1_00.tsv.gz",  sep='\t', compression='gzip', error_bad_lines=False, dtype='str')

b'Skipping line 85458: expected 15 fields, saw 22\nSkipping line 91161: expected 15 fields, saw 22\n'
b'Skipping line 166123: expected 15 fields, saw 22\n'
b'Skipping line 225458: expected 15 fields, saw 22\nSkipping line 229936: expected 15 fields, saw 22\nSkipping line 259297: expected 15 fields, saw 22\n'
b'Skipping line 284728: expected 15 fields, saw 22\nSkipping line 286334: expected 15 fields, saw 22\nSkipping line 293400: expected 15 fields, saw 22\nSkipping line 294415: expected 15 fields, saw 22\nSkipping line 308150: expected 15 fields, saw 22\nSkipping line 315022: expected 15 fields, saw 22\nSkipping line 315730: expected 15 fields, saw 22\nSkipping line 316071: expected 15 fields, saw 22\nSkipping line 326729: expected 15 fields, saw 22\n'
b'Skipping line 329101: expected 15 fields, saw 22\nSkipping line 333077: expected 15 fields, saw 22\nSkipping line 377031: expected 15 fields, saw 22\nSkipping line 389496: expected 15 fields, saw 22\nSkipping line 390486: expected 15 

In [36]:
df.dropna(inplace=True)

In [38]:
df.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [39]:
df = df[:100_000]
df = df[["review_body","star_rating"]]
df.head()

,review_body,star_rating
0,ok,5
1,"Perfect, even sturdier than the original!",5
2,"If the words, &#34;Cheap Chinese Junk&#34; com...",2
3,Exactly what I wanted and expected. Perfect fo...,5
4,I will look past the fact that they tricked me...,5


In [40]:
df.to_csv("cust_reviews_min.csv")

In [2]:
import pandas as pd
df = pd.read_csv("cust_reviews_min.csv")

In [3]:
df = df[ (df["star_rating"] == 1) | (df["star_rating"] == 5)]

In [4]:
df["star_rating"].value_counts()

5    62740
1     9664
Name: star_rating, dtype: int64

In [5]:
BERT_MODEL = "bert-base-uncased"
MAX_LENGTH = 250

In [14]:
#BERT
import transformers
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL)

# tokens = []
# for review in df["review_body"]:
#     token = tokenizer.encode(review, max_length=MAX_LENGTH, truncation=True)
#     tokens.append(token)


In [15]:
class ReviewDataset(Dataset):
    def __init__(self,reviews, targets, tokenizer):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = self.reviews[item]
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [30]:
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)

train_ds = ReviewDataset(reviews = df_train["review_body"], targets=df_train["star_rating"], tokenizer=tokenizer)
train_dl = DataLoader(train_ds, batch_size=32, num_workers=4)

test_ds = ReviewDataset(reviews = df_test["review_body"], targets=df_test["star_rating"], tokenizer=tokenizer)
test_dl = DataLoader(test_ds, batch_size=32, num_workers=4)


In [33]:

epochs = 10
device = "cpu"

def ret_model():
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", 
        num_labels = 2, 
        output_attentions = False, 
        output_hidden_states = False,
    )
    return model


optim = AdamW(model.parameters(), lr=1e-3, eps=1e-8)

total_steps = len(train_dl) * epochs
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=0, num_training_steps=total_steps)

for epoch in range(epochs):
    print("epoch ",epoch)
    model = ret_model()
    model.to(device)
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dl):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        lables = batch[2].to(device)

        model.zero_grad()
        loss, logits = model(input_ids, 
                        token_type_ids=None,
                        attention_mask = input_mask,
                        labels = labels)
        total_loss+= loss.item()
        loss.backward()
        optim.step()
        scheduler.step()
    avg_train_loss = total_loss / len(train_dl)
    print("avg_train_loss",avg_train_loss)
    model.eval()
    
    total_eval_acc = 0
    total_eval_loss = 0
    eval_steps = 0

    for batch in test_dl:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        lables = batch[2].to(device)

        with torch.no_grad():
            loss, logits = model(input_ids, 
                        token_type_ids=None,
                        attention_mask = input_mask,
                        labels = labels)

            total_eval_loss+= loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = lables.to("cpu").numpy()

        avg_val_acc = total_eval_acc / len(test_dl)
        print("Val acc",avg_val_acc)
        avg_val_loss = total_eval_loss / len(test_dl)
        print("val loss", avg_val_loss)


    
        


   




epoch  0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

KeyboardInterrupt: 